#Charger le jeu de données WMT14, Exploration, Pretraitement, Entrainnement et Sauvegarde du modèle

In [ ]:
# Importation des bibliothèques nécessaires
!pip install datasets transformers torch
from datasets import load_dataset
from transformers import MarianMTModel, MarianTokenizer, DataCollatorForSeq2Seq
from transformers import AdamW
from torch.utils.data import DataLoader
import torch

# Vérification de l'utilisation du GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Charger le jeu de données WMT14
#dataset = load_dataset("wmt14", "fr-en", split="train[:1%]")  # On utilise 1% pour tester rapidement
#train_test_split = dataset.train_test_split(test_size=0.1)
dataset = load_dataset("wmt14", "fr-en", split="train")
dataset = dataset.select(range(int(len(dataset) * 0.005))) # Select 0.5% of the data
train_test_split = dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]
val_dataset = train_test_split["test"]

# Charger le tokenizer et le modèle Marian pré-entraîné
model_name = "Helsinki-NLP/opus-mt-en-fr"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name).to(device)


# Prétraitement des données
def preprocess_function(batch):
    # Access the 'en' and 'fr' translations from the list of dictionaries
    en_translations = [example["en"] for example in batch["translation"]]
    fr_translations = [example["fr"] for example in batch["translation"]]

    inputs = tokenizer(en_translations, max_length=64, truncation=True, padding="max_length")
    targets = tokenizer(fr_translations, max_length=64, truncation=True, padding="max_length")
    inputs["labels"] = targets["input_ids"]
    return inputs


train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)

# Suppression des colonnes inutiles
train_dataset = train_dataset.remove_columns(["translation"])
val_dataset = val_dataset.remove_columns(["translation"])

# Convertir les données en format DataLoader
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=4, collate_fn=data_collator)
val_loader = DataLoader(val_dataset, batch_size=4, collate_fn=data_collator)

# Configuration de l'optimiseur
optimizer = AdamW(model.parameters(), lr=5e-5)

# Fonction d'entraînement
def train_epoch(model, loader, optimizer, device):
    model.train()
    total_loss = 0
    for batch in loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        total_loss += loss.item()
    return total_loss / len(loader)

# Fonction de validation
def validate_epoch(model, loader, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            total_loss += outputs.loss.item()
    return total_loss / len(loader)

# Boucle d'entraînement
epochs = 1
for epoch in range(epochs):
    train_loss = train_epoch(model, train_loader, optimizer, device)
    val_loss = validate_epoch(model, val_loader, device)
    print(f"Epoch {epoch + 1}: Train Loss = {train_loss:.4f}, Validation Loss = {val_loss:.4f}")

# Sauvegarder le modèle et le tokenizer
model.save_pretrained("trained-marian-wmt14")
tokenizer.save_pretrained("trained-marian-wmt14")


Resolving data files:   0%|          | 0/30 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/30 [00:00<?, ?it/s]

Map:   0%|          | 0/183764 [00:00<?, ? examples/s]

Map:   0%|          | 0/20419 [00:00<?, ? examples/s]

In [ ]:
print(dataset)

In [ ]:
from tabulate import tabulate

print(tabulate(dataset[:5]))

In [ ]:
print(train_dataset)

In [ ]:
print(val_dataset)

In [ ]:
print(dataset.features)

#Prédictions à partir de notre modèle

##Tester les Traductions

In [ ]:
from transformers import MarianTokenizer, MarianMTModel

# Charger le modèle et le tokenizer
tokenizer = MarianTokenizer.from_pretrained("trained-marian-wmt14")
model = MarianMTModel.from_pretrained("trained-marian-wmt14")

# Exemple de traduction
sentence = "The weather is nice today."
inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True)
translated_tokens = model.generate(**inputs)
translation = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)
print("Traduction :", translation)


/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Traduction : Aujourd'hui, la température est belle.


##Exemple de traduction en utilsant un Tableau de phrases

In [ ]:
from transformers import MarianTokenizer, MarianMTModel

# Charger le modèle et le tokenizer
tokenizer = MarianTokenizer.from_pretrained("trained-marian-wmt14")
model = MarianMTModel.from_pretrained("trained-marian-wmt14")

# Tableau de phrases à traduire (anglais vers français)
sentences = [
    "The weather is nice today.",
    "I love programming in Python.",
    "Artificial intelligence is transforming the world.",
    "Translation models are very useful.",
    "How are you today?"
]

# Traduire les phrases
def translate_sentences(sentences):
    translations = []
    for sentence in sentences:
        # Tokenization
        inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True)
        # Génération de la traduction
        translated_tokens = model.generate(**inputs)
        # Décodage
        translation = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)
        translations.append(translation)
    return translations

# Obtenir les traductions
translated_sentences = translate_sentences(sentences)

# Afficher les résultats
for i, (source, target) in enumerate(zip(sentences, translated_sentences)):
    print(f"Phrase {i + 1} (EN): {source}")
    print(f"Traduction (FR): {target}")
    print("-" * 40)


/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Phrase 1 (EN): The weather is nice today.
Traduction (FR): Aujourd'hui, la température est belle.
----------------------------------------
Phrase 2 (EN): I love programming in Python.
Traduction (FR): J'apprécie la programmation à Python.
----------------------------------------
Phrase 3 (EN): Artificial intelligence is transforming the world.
Traduction (FR): L' intelligence artificielle transforme le monde.
----------------------------------------
Phrase 4 (EN): Translation models are very useful.
Traduction (FR): Les modèles de traduction sont très utiles.
----------------------------------------
Phrase 5 (EN): How are you today?
Traduction (FR): Comment allez-vous aujourd'hui ?
----------------------------------------


##Exemple de traduction en batch

In [ ]:
def translate_sentences_in_batch(sentences, batch_size=4):
    translations = []
    for i in range(0, len(sentences), batch_size):
        batch = sentences[i:i + batch_size]
        inputs = tokenizer(batch, return_tensors="pt", truncation=True, padding=True, max_length=128)
        inputs = {k: v.to(model.device) for k, v in inputs.items()}  # Envoi des données au GPU
        translated_tokens = model.generate(**inputs)
        translations += [tokenizer.decode(tokens, skip_special_tokens=True) for tokens in translated_tokens]
    return translations

# Obtenir les traductions en batch
translated_sentences = translate_sentences_in_batch(sentences)
print(translated_sentences)

["Aujourd'hui, la température est belle.", "J'apprécie la programmation à Python.", "L' intelligence artificielle transforme le monde.", 'Les modèles de traduction sont très utiles.', "Comment allez-vous aujourd'hui ?"]


#Évaluation du modele avec le Score BLEU

In [ ]:
from sacrebleu import corpus_bleu
from transformers import MarianTokenizer, MarianMTModel

# Charger le modèle et le tokenizer
tokenizer = MarianTokenizer.from_pretrained("trained-marian-wmt14")
model = MarianMTModel.from_pretrained("trained-marian-wmt14")

# Tableau de phrases sources (en anglais)
sentences = [
    "The weather is nice today.",
    "I love programming in Python.",
    "Artificial intelligence is transforming the world.",
    "Translation models are very useful.",
    "How are you today?"
]

# Traductions de référence (en français) : Ce sont les traductions humaines idéales
references = [
    "Le temps est agréable aujourd'hui.",
    "J'adore programmer en Python.",
    "L'intelligence artificielle transforme le monde.",
    "Les modèles de traduction sont très utiles.",
    "Comment ça va aujourd'hui ?"
]

# Traduire les phrases sources avec le modèle
def translate_sentences(sentences):
    translations = []
    for sentence in sentences:
        inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True)
        translated_tokens = model.generate(**inputs)
        translation = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)
        translations.append(translation)
    return translations

# Obtenir les traductions générées
translations = translate_sentences(sentences)

# Calculer le score BLEU
bleu_score = corpus_bleu(translations, [references])

# Afficher les résultats
for i, (source, target, reference) in enumerate(zip(sentences, translations, references)):
    print(f"Phrase {i + 1} (EN): {source}")
    print(f"Traduction générée (FR): {target}")
    print(f"Référence (FR): {reference}")
    print("-" * 40)

print(f"\nScore BLEU : {bleu_score.score:.2f}")


/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Phrase 1 (EN): The weather is nice today.
Traduction générée (FR): Aujourd'hui, la température est belle.
Référence (FR): Le temps est agréable aujourd'hui.
----------------------------------------
Phrase 2 (EN): I love programming in Python.
Traduction générée (FR): J'apprécie la programmation à Python.
Référence (FR): J'adore programmer en Python.
----------------------------------------
Phrase 3 (EN): Artificial intelligence is transforming the world.
Traduction générée (FR): L' intelligence artificielle transforme le monde.
Référence (FR): L'intelligence artificielle transforme le monde.
----------------------------------------
Phrase 4 (EN): Translation models are very useful.
Traduction générée (FR): Les modèles de traduction sont très utiles.
Référence (FR): Les modèles de traduction sont très utiles.
----------------------------------------
Phrase 5 (EN): How are you today?
Traduction générée (FR): Comment allez-vous aujourd'hui ?
Référence (FR): Comment ça va aujourd'hui ?
---

# Interprétation du Score BLEU :
Le score est une mesure de similarité entre les traductions générées par le modèle et les traductions de référence (humaines). Il varie de 0 à 100.
Un score élevé indique une meilleure correspondance. Typiquement :

			<20 : Faible correspondance.
			20-40 : Traduction correcte, mais encore imparfaite.
			40-60 : Bonne traduction.
      60 : Traduction presque parfaite.